In [ ]:
import pandas as pd
import math

dict_intent = {}
list_clust_pass = []
for k in range(300):
    df = pd.read_csv("tf-idf-500-300/samples-per-k/" + str(k) + ".csv")
    df['x'] = [1] * len(df)
    if df.groupby('cluster').sum()['x'].values[0] >= math.floor(len(df)/2):
        list_clust_pass.append(k)
        if df['intent text'].values[0] not in dict_intent.keys():
            dict_intent[df['intent text'].values[0]] = 1
        else:
            dict_intent[df['intent text'].values[0]] = dict_intent.get(df['intent text'].values[0]) + 1

In [ ]:
import pandas as pd
import math

dict_intent = {}
list_clust_pass = []
for k in range(100):
    df = pd.read_csv("tf-idf-500-100/samples-per-k/" + str(k) + ".csv")
    if df.groupby('cluster').sum()['x'].values[0] >= math.floor(len(df)/2):
        list_clust_pass.append(k)
        if df['intent text'].values[0] not in dict_intent.keys():
            dict_intent[df['intent text'].values[0]] = 1
        else:
            dict_intent[df['intent text'].values[0]] = dict_intent.get(df['intent text'].values[0]) + 1

In [ ]:
import pandas as pd
import math

dict_intent = {}
list_clust_pass = []
for k in range(300):
    df = pd.read_csv("tf-idf-500-300/samples-per-k/" + str(k) + ".csv")
    list_clust_pass.append(k)
    if df['intent text'].values[0] not in dict_intent.keys():
        dict_intent[df['intent text'].values[0]] = 1
    else:
        dict_intent[df['intent text'].values[0]] = dict_intent.get(df['intent text'].values[0]) + 1

In [ ]:
len(dict_intent)

In [ ]:
import numpy as np

In [ ]:
dict_intent

In [ ]:
len(dict_intent)

In [ ]:
sum(dict_intent.values())

In [ ]:
list_clust_pass

In [ ]:
for k in list_clust_pass:
    df = pd.read_csv("res.csv")
    print(k, df[df['cluster'] == k]['mean score'].values[0])

In [ ]:
print( df.iloc[list_clust_pass]['mean score'].mean() )
print( df.iloc[list_clust_pass]['mean score'].std() )

In [ ]:
 df.iloc[list_clust_pass]['mean score'].mean() -  df.iloc[list_clust_pass]['mean score'].std() 

In [ ]:
print( df.iloc[np.setdiff1d(df.index.values, list_clust_pass)]['mean score'].mean() )
print( df.iloc[np.setdiff1d(df.index.values, list_clust_pass)]['mean score'].std() )

In [ ]:
df.iloc[np.setdiff1d(df.index.values, list_clust_pass)]['mean score'].mean() + df.iloc[np.setdiff1d(df.index.values, list_clust_pass)]['mean score'].std()

In [ ]:
df['x'] = [1 if x in list_clust_pass else 0 for x in df.cluster ]

In [ ]:
df['x'] = [1 if x >= 0.47 else 0 for x in df['mean score']]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.style.use('seaborn')

In [ ]:
# PLOT AXES
fig = plt.gcf()
fig.set_size_inches(20, 13)
plt.xticks(fontsize=12)
plt.yticks(fontsize=16)
plt.xlabel("cluster", fontsize=18)
plt.ylabel("mean score", fontsize=16)
ax = sns.barplot(data=df, x="cluster", y="mean score", linewidth=3, hue = 'x', hue='x')
rango = 3ange(0, 100 + 10, 10)
ax.set_xticks(rango)
for x in rango:
    ax.axvline(x, linestyle="-", color="#7f7f7f", linewidth=0.5)
plt.show()
plt.close()

In [ ]:
len(df[df['mean score'] >= 0.47]['intent'].unique())

In [ ]:
df['mean score'].hist()

In [ ]:
df[df['x'] >= 0.47].describe()['mean score']

In [ ]:
df[df['x'] >= 0.47].groupby(pd.Grouper(key='intent text')).count().reset_index().sort_values(by=['intent text'], ascending=True)

In [ ]:
df[df['x'] < 0.47].describe()['mean score']

---

In [ ]:
import pandas as pd
import numpy as np
import utils
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

In [ ]:
df = utils.leer_dpto('SOAD')
df_menus = utils.leer_menus_labels("menus-labels")
df_menus['idx'] = [i for i in range(0, len(df_menus))]
print(f"There are {len(df)} items in df")
print(f"There are {len(df_menus)} items in df_menus")

In [ ]:
samples = df.sample(n=383, random_state=42).index
samples

In [ ]:
# calculate the similarity and establish a treshold
BATCH_SIZE = 10000

tfidf = TfidfVectorizer(ngram_range=(1,3))
list_documents = df_menus['OracionLematizada'].values
docs_tfidf = tfidf.fit_transform(list_documents)

df['intent_idx'] = np.empty(len(df))
df['intent_sim'] = np.empty(len(df))
df['intent_text'] = np.empty(len(df))
for index in range(0, df.shape[0], BATCH_SIZE):
    batch = df.iloc[index:min(index+BATCH_SIZE,df.shape[0]),:]
    list_query = batch['OracionLematizada'].values
    query_tfidf = tfidf.transform(list_query)
    cosineSimilarities = cosine_similarity(docs_tfidf, query_tfidf).T
    df.iloc[batch.index.values, df.columns.get_loc('intent_idx')] = [np.argmax(sim) for sim in cosineSimilarities]
    df.iloc[batch.index.values, df.columns.get_loc('intent_sim')] = [np.max(sim) for sim in cosineSimilarities]
    df['intent_idx'] = df['intent_idx'].astype(int)
    df.iloc[batch.index.values, df.columns.get_loc('intent_text')] = df_menus.iloc[df.iloc[batch.index.values, df.columns.get_loc('intent_idx')]]['OracionLematizada']
    print(index)

In [ ]:
df.head()

In [ ]:
df.iloc[samples].sample(5)

In [ ]:
df.iloc[samples]